In [ ]:
import numpy as np
from paus_utils import *
from astropy.table import Table
from load_paus_cat import paus_flux_units

from jpasLAEs.utils import mag_to_flux

from LAE_selection_method import *

In [ ]:
import seaborn as sns

corr_dir = '/home/alberto/almacen/PAUS_data/LF_corrections'
r_bins = np.linspace(17, 24, 200 + 1)
L_bins = np.linspace(40, 47, 200 + 1)
puri2d = np.load(f'{corr_dir}/puri2D_foo.npy')

fig, ax = plt.subplots(figsize=(7, 7))

sns.heatmap(puri2d.T, vmin=0, vmax=1, ax=ax,
            cmap='Spectral', rasterized=True)

yticks = np.arange(len(r_bins))[::20]
ax.set_yticks(yticks)
ax.set_yticklabels(f'{s:0.1f}' for s in r_bins[yticks])

xticks = np.arange(len(L_bins))[::20]
ax.set_xticks(yticks)
ax.set_xticklabels(f'{s:0.1f}' for s in L_bins[xticks])

ax.set(xlim=(80, 170), ylim=(200, 15))

plt.show()

In [ ]:
tab = Table.read('/home/alberto/almacen/PAUS_data/catalogs/EDR_1_v2.fits').to_pandas()

cat = {} # Initialize catlalog dict

cat['ref_id'] = np.array(tab['ref_id'])
cat['photoz'] = np.array(tab['zb'])
cat['photoz_odds'] = np.array(tab['odds'])
cat['ra'] = np.array(tab['ra'])
cat['dec'] = np.array(tab['dec'])
cat['type'] = np.array(tab['type'])
cat['zspec'] = np.array(tab['zspec'])
cat['r_mag'] = np.array(tab['rmag'])

# Flux units have to be converted to erg s^-1 A^-1
cat['flx'] = paus_flux_units(tab.to_numpy()[:, 7 : 7 + 40],
                                w_central[:-6]).T
cat['err'] = paus_flux_units(tab.to_numpy()[:, 7 + 40 : 7 + 40 + 40],
                                w_central[:-6]).T

N_sources = len(cat['ref_id'])

# Add provisional BBs
cat['flx'] = np.vstack([cat['flx'],
                        mag_to_flux(tab.to_numpy()[:, -5:].T, w_central[-6:-1].reshape(-1,1)),
                        np.zeros(N_sources)])
cat['err'] = np.vstack([cat['err'],
                        np.zeros((6, N_sources))])

In [ ]:
def load_this_cat():
    tab = Table.read('/home/alberto/almacen/PAUS_data/catalogs/LightCone_mock.fits').to_pandas().to_numpy()
    mock_size = len(tab)

    cat_fraction = 0.5 # Load only a fraction of the mock
    np.random.seed(1312)
    sel = np.random.randint(0, mock_size, size=int(mock_size * cat_fraction))
    
    cat = {}

    cat['flx'] = mag_to_flux(tab[sel, 11 : 11 + 40],
                                    w_central[:-6]).T

    # Add provisional BBs
    cat['flx'] = np.vstack([cat['flx'],
                            mag_to_flux(tab[sel, -5:].T,
                                        w_central[-6:-1].reshape(-1,1)),
                            np.zeros(len(sel))])

    cat['r_mag'] = flux_to_mag(cat['flx'][-4], w_central[-4])
    cat['i_mag'] = flux_to_mag(cat['flx'][-3], w_central[-3])
    mag_mask = np.array(cat['r_mag'] < 24) & np.array(cat['i_mag'] < 23)
    cat['flx'] = cat['flx'][:, mag_mask]
    cat['r_mag'] = cat['r_mag'][mag_mask]

    nominal_errs = mag_to_flux(23, w_central) / 3
    cat['err'] = np.ones_like(cat['flx']) * nominal_errs.reshape(-1, 1)

    # Apply errs
    cat['flx'] += cat['err'] * np.random.normal(size=cat['flx'].shape)

    cat['zspec'] = np.array(tab[:, 4])[sel][mag_mask]

    return cat

cat = load_this_cat()

N_sources = cat['flx'].shape[1]
print(f'N_sources = {N_sources}')

In [ ]:
# Load QSO mock

# qso_mock_name = 'QSO_PAUS_LAES_2'
qso_mock_name = 'LAE_12.5deg_z2.55-5_PAUS_0'
# qso_mock = pd.read_csv(f'/home/alberto/almacen/Source_cats/{qso_mock_name}/data325350.csv')
qso_mock = pd.read_csv(f'/home/alberto/almacen/Source_cats/{qso_mock_name}/data3.csv')
qso_mock

cat = {}

cat['r_mag'] = flux_to_mag(np.array(qso_mock['r']), w_central[-4])

# Flux units have to be converted to erg s^-1 A^-1
cat['flx'] = qso_mock.to_numpy()[:, 1 : 1 + 46].T
cat['err'] = cat['flx'] / 10

N_sources = len(cat['r_mag'])
print(f'N_sources = {N_sources}')

In [ ]:
nb_min, nb_max = 0, 12
print(f'Searching for LAEs at {z_NB(nb_min):0.2f} < z < {z_NB(nb_max):0.2f}')

# Estimate continuum
cont_est, cont_err = estimate_continuum(cat['flx'], cat['err'],
                                        IGM_T_correct=True, N_nb=6)
cont_est_other, cont_err_other = estimate_continuum(cat['flx'], cat['err'],
                                                    IGM_T_correct=False,
                                                    N_nb=6)
# Identify NB excess
is_line = is_there_line(cat['flx'], cat['err'], cont_est, cont_err,
                        ew0min=30)
lya_lines = identify_lines(is_line, cat['flx'], cont_est, first=True)
other_lines = identify_lines(is_line, cat['flx'], cont_est_other, first=False)

# Estimate redshift (z_Arr)
z_Arr = z_NB(lya_lines)

snr = np.empty(N_sources)
for src in range(N_sources):
    l = lya_lines[src]
    snr[src] = cat['flx'][l, src] / cat['err'][l, src]

nb_mask = (lya_lines >= nb_min) & (lya_lines <= nb_max)
snr_mask = (snr >= 6)

nice_lya_mask = snr_mask & nb_mask

nice_lya, _, _ = nice_lya_select(lya_lines, other_lines, cat['flx'],
                                 z_Arr, mask=nice_lya_mask)

print(f'N nice_lya = {sum(nice_lya)}')

In [ ]:
qso_lines = [1025.7220, 1397.61, 1549.48, 1908.73, 2799.12, 2326, 3727]
qso_lines_name = ['LyB', 'SiIV', 'CIV', 'CIII', 'MgII', 'CII', 'OII']

selection = nice_lya

for i, src in enumerate(np.random.permutation(np.where(selection)[0])):
    if i == 10:
        break

    print(f'z_NB = {z_Arr[src]:0.2f}, ',
          f'r = {cat["r_mag"][src]:0.2f}')

    fig, ax = plt.subplots(figsize=(8, 3.5))

    plot_PAUS_source(cat['flx'][:, src], cat['err'][:, src],
                     ax=ax, plot_BBs=False, set_ylim=False)

    ax.axvline(w_central[lya_lines[src]], ls='--', c='r')

    # Show other detected lines
    for l in np.where(is_line[:, src])[0]:
        ax.axvline(w_central[l], ls=':', c='dimgray', zorder=99)

    for name, w in zip(qso_lines_name, qso_lines):
        qso_obs_w = w * (1 + z_Arr[src])
        if (4400 > qso_obs_w) | (qso_obs_w > 8600):
            continue
        ax.axvline(qso_obs_w, linestyle=':', color='orange')
        ypos = cat['flx'][:40, src].min() * 1e17
        ax.text(qso_obs_w, ypos, name, color='k')

    ax.plot(w_central[:30], cont_est[:30, src] * 1e17)
    
    plt.show()

In [ ]:
path_to_fil_props = '/home/alberto/almacen/PAUS_data/Filter_properties.csv'
filter_properties = pd.read_csv(path_to_fil_props)

In [ ]:
fig, ax = plt.subplots()

for i, nb in enumerate([0, 5, 10, 15, 20]):
    ax.plot(flux_to_mag(cat['flx'][nb], w_central[nb]),
            cat['flx'][nb] / cat['err'][nb],
            ls='', marker='.', ms=3, alpha=0.8,
            color=f'C{i}')

    # Dummy for legend
    ax.plot([], [], c=f'C{i}',
            ls='', marker='.', ms=10,
            label=filter_properties['name'][nb])

ax.axhline(5, ls='--', c='k')

ax.legend()

ax.set(yscale='log',
       xlabel='magAB',
       ylabel='S/N',
       xlim=(17, 26), ylim=(1, 100))

plt.show()